<H1>Connecting to a remote database using ODM2API and adding time series values using SQLAlchemy</H1>
<p> 1) load python classes IPython display classes and widgets </p>

In [36]:
import sys
import os
import getpass
import csv
from IPython.display import display, HTML
import ipywidgets as widgets


<p>2) Next load ODM2api database connection class and models</p>

In [37]:
from odm2api.ODMconnection import dbconnection
from odm2api.ODM2.models import *

<p>3) Now create text widgets and a container for database connection info.</p>

In [38]:
#print("Enter your ODM2 username") 
container = widgets.Box() # would be nice If I could get a container to hold the 
# user name and password prompt, getpass doesn't seem to play well with the other 
# widgets though
username_text = widgets.Text(
    value='', placeholder='Enter username',
    description='', disabled=False)
username_output_text = widgets.Text(
    value='', placeholder='Enter username',
    description='Username',disabled=False)
database_address_text = widgets.Text(
    value='', placeholder='Enter database address',
    description='',disabled=False)
database_address_output_text = widgets.Text(
    value='',placeholder='Enter database address',
    description='database address',disabled=False)
database_text = widgets.Text(
    value='', placeholder='Enter database name',
    description='', disabled=False)
database_output_text = widgets.Text(
    value='', placeholder='Enter database name',
    description='database name', disabled=False)
def bind_username_to_output(sender):
    username_output_text.value = username_text.value
def bind_database_address_to_output(sender):
    database_address_output_text.value = database_address_text.value
def bind_database_to_output(sender):
    database_output_text.value = database_text.value     
    
def login(sender):
    #print('Database address : %s, Username: %s, database name: %s' % (
    #    database_address_text.value, username_text.value, database_text.value))
    container.close()    
    
username_text.on_submit(bind_username_to_output)
login_btn = widgets.Button(description="Login")
login_btn.on_click(login)
container.children = [username_text,database_address_text, database_text, login_btn]
container
#username_text


<p>4) Now prompt the user for their password. </p>

In [39]:
print("enter your password: ")
p = getpass.getpass()


enter your password: 
········


<p> 5) Now create a connection to the database </p>

In [40]:
session_factory = dbconnection.createConnection('postgresql', database_address_text.value, database_text.value, 
                                                username_text.value, p) 

<p> 6) Create a database session and use SQLALchemy to pull some initial records.</p>
<p>    a) find the feature actions for the sampling feature.</p>
<p>    b) narrow the feature actions based on the method of the action.</p>


In [41]:
DBSession = session_factory.getSession()
#var = DBSession.query(Results).get(16650) #LGW2B well temperature
#print(var)
#find the feature actions for 
feature_actions = DBSession.query(FeatureActions).join(SamplingFeatures).filter(
    SamplingFeatures.SamplingFeatureName.ilike('%LGW2B%'))

feature_action_depth = feature_actions.join(Actions).join(Methods).filter(Methods.MethodName.ilike('%depth%')).one()
feature_action_surface = feature_actions.join(Actions).join(Methods).filter(Methods.MethodName.ilike('%surface%')).one()

depth_results = DBSession.query(Results).join(FeatureActions).filter_by(
    FeatureActionID=feature_action_depth.FeatureActionID)

surface_results = DBSession.query(Results).join(FeatureActions).filter_by(
    FeatureActionID=feature_action_surface.FeatureActionID)

surface_result_id = []
depth_resultid=[]
print("Results for Pressure transducer at depth")
for depth_result in depth_results:
    print(depth_result)
    depth_resultid.append(depth_result.ResultID)
    
print("Results for Pressure transducer at surface")
for surface_result in surface_results:
    print(surface_result)
    surface_result_id.append(surface_result.ResultID)
#sf = DBSession.query(FeatureActions).get(774)
#print(feature_action)

#results = DBSession.query(Results).join(FeatureActions).filter_by(FeatureActionID=featureaction.FeatureActionID)
#for result in results:
#    print result



Results for Pressure transducer at depth
<Results('16644', 'ce603ba9-d03e-4a08-825d-a1a82e71bf7a', 'Time series coverage', '1', '1')>
<Results('16650', 'e1a696ff-97fb-485a-ac30-b6418bd78651', 'Time series coverage', '1', '1')>
<Results('16651', '44c1848e-5de0-4b86-ab6d-27d62efb996d', 'Time series coverage', '1', '1')>
Results for Pressure transducer at surface
<Results('16652', 'e890bd26-5321-43b5-9b40-ebd9d166f9c4', 'Time series coverage', '1', '1')>
<Results('16653', '94fdd54f-ffd7-4384-b691-8bbe6f369722', 'Time series coverage', '1', '1')>
<Results('16645', '0e95dfe0-0979-4993-8db8-3985e341bc41', 'Time series coverage', '1', '1')>
<Results('16668', 'd968d77c-f13c-478e-b5db-759f2a223767', 'Time series coverage', '3', '1')>


<p> 7) Open a local file with time series data we want to load into the database. <p> 

In [34]:
infile = open('./data/LGW2B/LGW2BWell_8-13-2016to8-13-2016.csv', 'rt')
reader = csv.reader(infile)
i=0

for line in reader:
    if i<5:
        i+=1
        print(line)
    else:
        break
infile.close()

['Plot Title: LGW2BWell', 'Abs Pres kPa (LGR S/N: 10434423, SEN S/N: 10434423)', 'Temp C (LGR S/N: 10434423 SEN S/N: 10434423)', 'Abs Pres kPa (LGR S/N: 10434421 SEN S/N: 10434421)', '']
['Date Time GMT-04:00', 'Abs Pres', 'Temp C', 'Abs Pres Barom- psi', 'Depth-meters']
['8/13/2016 0:00', '112.712', '20.901', '93.381', '1.971267685']
['8/13/2016 0:15', '112.728', '20.901', '93.381', '1.972899276']
['8/13/2016 0:30', '112.712', '20.901', '93.365', '1.972899276']


8) check if the value are already in the database.

In [35]:
#valueids = []
for surface_result in surface_results:
    resultid = surface_result.ResultID
    print(surface_result)
    time_series_values = DBSession.query(TimeSeriesResultValues).filter(
        TimeSeriesResultValues.ValueDateTime >= '8/13/2016 0:00').join(TimeSeriesResults).filter_by(
        ResultID=surface_result.ResultID)
    print("values for this result:")
    print(time_series_values.count())
    for time_series_value in time_series_values:
        print(time_series_value.ValueID)
        #valueids.append(time_series_value.ValueID)

<Results('16652', 'e890bd26-5321-43b5-9b40-ebd9d166f9c4', 'Time series coverage', '1', '1')>
values for this result:
0
<Results('16653', '94fdd54f-ffd7-4384-b691-8bbe6f369722', 'Time series coverage', '1', '1')>
values for this result:
0
<Results('16645', '0e95dfe0-0979-4993-8db8-3985e341bc41', 'Time series coverage', '1', '1')>
values for this result:
0
<Results('16668', 'd968d77c-f13c-478e-b5db-759f2a223767', 'Time series coverage', '3', '1')>
values for this result:
0


<p> 9) Using SQLAlchemy load some of the new data records. </p>

In [25]:
#  well surface pressure

#well_surface_pressure_time_series_result = DBSession.query(TimeSeriesResults).filter(
#    TimeSeriesResults.ResultID.in_(surface_result_id))
#for wsptsr in well_surface_pressure_time_series_result:
#    print(wsptsr)
infile = open('./data/LGW2B/LGW2BWell_8-13-2016to8-13-2016.csv', 'rt')
reader = csv.reader(infile)
i=0
valueids = []
for line in reader:
    if i<10:
        i+=1
        if i<3:
            print('header')
            print(line)
            continue
    else:
        break
    for surface_result in surface_results:
        variable_id = surface_result.VariableID
        variable = DBSession.query(Variables).get(variable_id)
        #print(variable)
        #print(variable.VariableCode)
        units_id = surface_result.UnitsID
        units = DBSession.query(Units).get(units_id)
        #print(units)
        if variable.VariableCode == 'Abs Pres':
            tsrv = TimeSeriesResultValues(ResultID=surface_result.ResultID, DataValue=line[1],
                                          ValueDateTime=line[0], ValueDateTimeUTCOffset=4, 
                                          CensorCodeCV='Not censored',QualityCodeCV='Good',
                                          TimeAggregationInterval=15, 
                                          TimeAggregationIntervalUnitsID=5)
            DBSession.add(tsrv)
            DBSession.commit()
            valueids.append(tsrv.ValueID)
infile.close()
print(valueids)

header
['Plot Title: LGW2BWell', 'Abs Pres kPa (LGR S/N: 10434423, SEN S/N: 10434423)', 'Temp C (LGR S/N: 10434423 SEN S/N: 10434423)', 'Abs Pres kPa (LGR S/N: 10434421 SEN S/N: 10434421)', '']
header
['Date Time GMT-04:00', 'Abs Pres', 'Temp C', 'Abs Pres Barom- psi', 'Depth-meters']
[11170223L, 11170224L, 11170225L, 11170226L, 11170227L, 11170228L, 11170229L, 11170230L]


10) check that the new values are in the database. 

In [26]:
for surface_result in surface_results:
    resultid = surface_result.ResultID
    print(surface_result)
    time_series_values = DBSession.query(TimeSeriesResultValues).filter(
        TimeSeriesResultValues.ValueDateTime >= '8/13/2016 0:00').join(TimeSeriesResults).filter_by(
        ResultID=surface_result.ResultID)
    print("values for this result:")
    print(time_series_values.count())
    for time_series_value in time_series_values:
        print(time_series_value.ValueID)

<Results('16652', 'e890bd26-5321-43b5-9b40-ebd9d166f9c4', 'Time series coverage', '1', '1')>
values for this result:
0
<Results('16653', '94fdd54f-ffd7-4384-b691-8bbe6f369722', 'Time series coverage', '1', '1')>
values for this result:
0
<Results('16645', '0e95dfe0-0979-4993-8db8-3985e341bc41', 'Time series coverage', '1', '1')>
values for this result:
8
11170223
11170224
11170225
11170226
11170227
11170228
11170229
11170230
<Results('16668', 'd968d77c-f13c-478e-b5db-759f2a223767', 'Time series coverage', '3', '1')>
values for this result:
0


11) Finally delete the records we just created so this script can be run again. 

In [27]:
time_series_values = DBSession.query(TimeSeriesResultValues).filter(
        TimeSeriesResultValues.ValueID.in_(valueids))
for time_series_value in time_series_values:
    DBSession.delete(time_series_value)
DBSession.commit()